In [141]:
print("This is all about the Fake News Detection model..")

This is all about the Fake News Detection model..


In [ ]:
# scrapping of Fakenews column from the dainik bhashkar 
# we will procecced to mannual scapping since the api is unablabe to access 
# using the beautifull soup and seleinium 


In [48]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver #for the scrolldown of web page.
import time
import pickle


In [ ]:
from selenium.webdriver.edge.service import Service

# Replace 'path_to_webdriver' with the path to your downloaded Edge WebDriver executable
service = Service(executable_path="D:\OneDrive\Documents\edgedriver_win64\msedgedriver.exe")
driver = webdriver.Edge(service=service)
url = "https://www.bhaskar.com/no-fake-news/"
driver.get(url)


# Scroll multiple times to load more content
scroll_pause_time = 5  # Pause between scrolls
scroll_up_amount=1000
time.sleep(2)


num_scrolls = 2
driver.execute_script("window.scrollTo(0, 1000);")
time.sleep(2)
for i in range(num_scrolls):
     # Scroll to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(scroll_pause_time)  # Wait for new content to load
    
    # Scroll up by a specific pixel amount
    current_position = driver.execute_script("return window.pageYOffset;")
    target_position = current_position - scroll_up_amount
    driver.execute_script(f"window.scrollTo(0, {target_position});")
    time.sleep(scroll_pause_time)  # Wait for content to load

In [ ]:

soup = BeautifulSoup(driver.page_source, 'html.parser')
# print(soup)
articles_heading = soup.find_all('li', class_='c7ff6507 db9a2680')
print(len(articles_heading))
links=[]

for articles in articles_heading:
    
    url=articles.find('a').get('href')
    links.append("https://www.bhaskar.com"+url)

396


In [51]:
links

['https://www.bhaskar.com/no-fake-news/news/donald-trump-pandit-white-house-viral-photo-exposed-133932344.html',
 'https://www.bhaskar.com/no-fake-news/news/donald-trump-pandit-white-house-viral-photo-exposed-133932344.html',
 'https://www.bhaskar.com/no-fake-news/news/donald-trump-victory-speech-viral-video-pm-narendra-modi-us-election-133921224.html',
 'https://www.bhaskar.com/no-fake-news/news/donald-trump-victory-speech-viral-video-pm-narendra-modi-us-election-133921224.html',
 'https://www.bhaskar.com/no-fake-news/news/after-the-india-china-agreement-did-the-soldiers-of-both-the-countries-chant-jai-shri-ram-on-the-lac-know-the-truth-of-the-viral-video-133867474.html',
 'https://www.bhaskar.com/no-fake-news/news/after-the-india-china-agreement-did-the-soldiers-of-both-the-countries-chant-jai-shri-ram-on-the-lac-know-the-truth-of-the-viral-video-133867474.html',
 'https://www.bhaskar.com/no-fake-news/news/did-the-chinese-president-refuse-to-shake-hands-with-pm-modi-know-the-truth-of

In [53]:
# now we have links to access the content of the news . 
# now need to extract that .

news=[]
lable_content=[]
for url in links:
    response=requests.get(url)
    soup=BeautifulSoup(response.text,'html.parser')
    article = soup.find('article',class_="bf64dc76")
    if article:
        paragraphs = article.find_all('p')
        lable_content.append(paragraphs[-2].get_text())
        obj=""
        for p in paragraphs:
            obj=obj+p.get_text()
            
        # print(obj)
        news.append(obj)
    # break


In [54]:
print(len(links))
print(len(news))

print(len(lable_content))

396
396
396


In [55]:
df=pd.DataFrame({"links":links,"news":news,"label":lable_content})
with open('dataframe.pkl', 'wb') as file:
    pickle.dump(df, file)

In [102]:
df=df[['links','news']]



In [105]:
    fake_keywords = ["भ्रामक", "असत्य", "गलत", "फर्जी"]
    def label_article(text):
        return "fake" if any(pattern in text for pattern in fake_keywords) else "unlabeled"

    
    df["label"]=df['news'].apply(label_article) 

In [115]:
# df["label"]
df['label'] = df['label'].replace("unlabeled", "not fake")
df['label']

0          fake
1          fake
2          fake
3          fake
4          fake
         ...   
391    not fake
392    not fake
393    not fake
394    not fake
395    not fake
Name: label, Length: 396, dtype: object

In [116]:
from transformers import BertTokenizer, BertModel
import torch

# Load mBERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def get_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():  # Disable gradients to speed up
        outputs = model(**inputs)
    # Use mean pooling on the last hidden state
    embeddings = outputs.last_hidden_state.mean(dim=1).squeeze()
    return embeddings

# Apply to each news article
df['embedding'] = df['news'].apply(get_embeddings)


C:\Users\HP-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [117]:
df["embedding"]

0      [tensor(0.0049), tensor(-0.3701), tensor(0.509...
1      [tensor(0.0049), tensor(-0.3701), tensor(0.509...
2      [tensor(0.0162), tensor(-0.2782), tensor(0.411...
3      [tensor(0.0162), tensor(-0.2782), tensor(0.411...
4      [tensor(0.0060), tensor(-0.4367), tensor(0.428...
                             ...                        
391    [tensor(0.0725), tensor(-0.2730), tensor(0.306...
392    [tensor(-0.1211), tensor(-0.2924), tensor(0.10...
393    [tensor(0.0029), tensor(-0.2508), tensor(0.463...
394    [tensor(-0.0801), tensor(-0.3654), tensor(0.59...
395    [tensor(-0.1328), tensor(-0.2500), tensor(0.43...
Name: embedding, Length: 396, dtype: object

In [118]:
embedding_matrix = np.stack(df['embedding'].values)
embedding_matrix

array([[ 0.00492613, -0.37013513,  0.5089685 , ...,  0.3846812 ,
         0.03014933, -0.20442504],
       [ 0.00492613, -0.37013513,  0.5089685 , ...,  0.3846812 ,
         0.03014933, -0.20442504],
       [ 0.01619319, -0.27822736,  0.4110629 , ...,  0.29438803,
        -0.00066932, -0.1546514 ],
       ...,
       [ 0.00291384, -0.2507855 ,  0.46357644, ...,  0.29952186,
        -0.11585846, -0.13697852],
       [-0.0800753 , -0.3654394 ,  0.5903075 , ...,  0.27806515,
         0.24303772, -0.22705522],
       [-0.13276997, -0.2500233 ,  0.43708873, ...,  0.12718393,
         0.14249454, -0.12156437]], dtype=float32)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(embedding_matrix, df['label'], test_size=0.2, random_state=42)

# Train a Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

        fake       0.85      1.00      0.92        58
    not fake       1.00      0.55      0.71        22

    accuracy                           0.88        80
   macro avg       0.93      0.77      0.81        80
weighted avg       0.89      0.88      0.86        80



In [120]:
# Predict probabilities for the "not fake" articles
df_not_fake = df[df['label'] == 'not fake']
not_fake_embeddings = np.stack(df_not_fake['embedding'].values)
not_fake_probs = model.predict_proba(not_fake_embeddings)
confidence_threshold = 0.8
high_confidence_real = (not_fake_probs[:, model.classes_.tolist().index('not fake')] > confidence_threshold)

# Label high-confidence "not fake" predictions as "real"
df.loc[df_not_fake.index[high_confidence_real], 'label'] = 'real'



In [121]:
df['label']

0          fake
1          fake
2          fake
3          fake
4          fake
         ...   
391        real
392    not fake
393        real
394        real
395    not fake
Name: label, Length: 396, dtype: object

In [ ]:

X_train, X_test, y_train, y_test = train_test_split(
    np.stack(df['embedding'].values),
    df['label'],
    test_size=0.2,
    random_state=42
)
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

        fake       0.85      1.00      0.92        58
    not fake       0.00      0.00      0.00        12
        real       0.83      1.00      0.91        10

    accuracy                           0.85        80
   macro avg       0.56      0.67      0.61        80
weighted avg       0.72      0.85      0.78        80



C:\Users\HP-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\HP-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344

In [126]:
df['label'].value_counts()

label
fake        266
not fake     65
real         65
Name: count, dtype: int64

In [ ]:
# Steps to Fine-Tune BERT with the Updated Dataset
# Preprocess the Data: Convert your textual data into the format required by BERT.
# Prepare the Dataset: Convert your text data into tokenized input suitable for BERT.
# Fine-Tune BERT: Fine-tune the BERT model with your labeled data (now containing only "fake" and "real" labels).
# Evaluate the Model: Check the accuracy of the fine-tuned model.

In [127]:
# Convert "not fake" to "real"
df['label'] = df['label'].replace("not fake", "real")

# Ensure that we now only have "fake" and "real" labels
print(df['label'].value_counts())


label
fake    266
real    130
Name: count, dtype: int64


In [129]:
df

,links,news,label,embedding
0,https://www.bhaskar.com/no-fake-news/news/dona...,बुधवार यानी 6 नवंबर को अमेरिकी राष्ट्रपति चुना...,fake,"[tensor(0.0049), tensor(-0.3701), tensor(0.509..."
1,https://www.bhaskar.com/no-fake-news/news/dona...,बुधवार यानी 6 नवंबर को अमेरिकी राष्ट्रपति चुना...,fake,"[tensor(0.0049), tensor(-0.3701), tensor(0.509..."
2,https://www.bhaskar.com/no-fake-news/news/dona...,अमेरिका में बुधवार को आए राष्ट्रपति चुनाव के न...,fake,"[tensor(0.0162), tensor(-0.2782), tensor(0.411..."
3,https://www.bhaskar.com/no-fake-news/news/dona...,अमेरिका में बुधवार को आए राष्ट्रपति चुनाव के न...,fake,"[tensor(0.0162), tensor(-0.2782), tensor(0.411..."
4,https://www.bhaskar.com/no-fake-news/news/afte...,भारत-चीन के बीच वास्तविक नियंत्रण रेखा (LAC) प...,fake,"[tensor(0.0060), tensor(-0.4367), tensor(0.428..."
...,...,...,...,...
391,https://www.bhaskar.com/no-fake-news/news/imra...,क्या हो रहा है वायरल : पाकिस्तान के गुजरांवाला...,real,"[tensor(0.0725), tensor(-0.2730), tensor(0.306..."
392,https://www.bhaskar.com/no-fake-news/news/yout...,क्या हो रहा है वायरल : गुजरात के मोरबी में बीत...,real,"[tensor(-0.1211), tensor(-0.2924), tensor(0.10..."
393,https://www.bhaskar.com/no-fake-news/news/afte...,क्या हो रहा है वायरल : भारतीय मूल के ऋषि सुनक ...,real,"[tensor(0.0029), tensor(-0.2508), tensor(0.463..."
394,https://www.bhaskar.com/no-fake-news/news/musl...,क्या हो रहा है वायरल : सोशल मीडिया पर एक वीडिय...,real,"[tensor(-0.0801), tensor(-0.3654), tensor(0.59..."


In [ ]:
from sklearn.model_selection import train_test_split

# Split the dataset into train and validation
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Define a function to create PyTorch datasets
# This is required for the fine tunning of the model .
class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, df, tokenizer):
        self.encodings = tokenizer(df['news'].tolist(), truncation=True, padding=True, max_length=512)
        self.labels = df['label'].apply(lambda x: 1 if x == 'real' else 0).tolist()  # 1 for 'real', 0 for 'fake'

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = NewsDataset(train_df, tokenizer)
val_dataset = NewsDataset(val_df, tokenizer)


In [133]:
# train_dataset[0]

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments


model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Output directory to save model
    evaluation_strategy="epoch",     # Evaluate the model after each epoch
    learning_rate=2e-5,              # Learning rate
    per_device_train_batch_size=8,   # Batch size for training
    per_device_eval_batch_size=8,    # Batch size for evaluation
    num_train_epochs=3,              # Number of training epochs
    weight_decay=0.01,               # Weight decay to avoid overfitting
)

# Initialize the Trainer API with the model, training arguments, and dataset
trainer = Trainer(
    model=model,                         # The model to train
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=val_dataset             # Validation dataset
)

# Train the model
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\HP-PC\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/120 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.4566064774990082, 'eval_runtime': 36.9424, 'eval_samples_per_second': 2.166, 'eval_steps_per_second': 0.271, 'epoch': 1.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.48033007979393005, 'eval_runtime': 37.024, 'eval_samples_per_second': 2.161, 'eval_steps_per_second': 0.27, 'epoch': 2.0}


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.4491804242134094, 'eval_runtime': 36.1922, 'eval_samples_per_second': 2.21, 'eval_steps_per_second': 0.276, 'epoch': 3.0}
{'train_runtime': 1823.0321, 'train_samples_per_second': 0.52, 'train_steps_per_second': 0.066, 'train_loss': 0.4389945983886719, 'epoch': 3.0}


TrainOutput(global_step=120, training_loss=0.4389945983886719, metrics={'train_runtime': 1823.0321, 'train_samples_per_second': 0.52, 'train_steps_per_second': 0.066, 'total_flos': 249429280481280.0, 'train_loss': 0.4389945983886719, 'epoch': 3.0})

In [137]:
# Evaluate the model on the validation set
eval_results = trainer.evaluate()
print(eval_results)


  0%|          | 0/10 [00:00<?, ?it/s]

{'eval_loss': 0.4491804242134094, 'eval_runtime': 41.4147, 'eval_samples_per_second': 1.932, 'eval_steps_per_second': 0.241, 'epoch': 3.0}


In [ ]:

def predict(text):
    # Preprocess the input text by tokenizing it
    encoding = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)
    
    # Perform inference using the fine-tuned model
    with torch.no_grad():  # No need to compute gradients during inference
        outputs = model(**encoding)
    
    logits = outputs.logits
    
    # Get the predicted class (0 or 1)
    predicted_class = torch.argmax(logits, dim=-1)
    
    # Return the prediction as 'real' or 'fake'
    if predicted_class == 1:
        return "real"
    else:
        return "fake"

# Example usage
article = "क्या सलमान खान ने लॉरेंस को जवाब दिया है? दरअसल, पिछले दिनों हुए एनसीपी नेता बाबा सिद्दीकी के मर्डर की जिम्मेदारी लॉरेंस गैंग ने ली है। सलमान खान पहले से ही लॉरेंस गैंग के निशाने पर हैं।वेरिफाइड एक्स यूजर डॉ. शीतल यादव ने अपने ट्वीट में लिखा-सलमान खान ने साफ शब्दों में लॉरेंस को जवाब दिया है। 'भिड़ोगे तो मिट जाओगे'। (अर्काइव ट्वीट)देखें ट्वीट:खबर लिखे जाने तक डॉ. शीतल यादव के इस ट्वीट को 19 हजार लोग लाइक कर चुके थे वहीं, 3300 लोगों ने इसे रीपोस्ट किया था। एक्स पर डॉ. शीतल यादव को 24 हजार यूजर्स फॉलो करते हैं।एक्स हैंडल, बेवजह-के-ख़याल ने भी वायरल वीडियो को ट्वीट करते हुए लिखा- 'सलमान खान का पैगाम लॉरेंस बिश्नोई को'। (अर्काइव लिंक)देखें ट्वीट:वहीं, वेरिफाइड एक्स यूजर अभिषेक गुप्ता ने अपने ट्वीट में लिखा-सलमान खान ने अब लॉरेंस बिश्नोई को धमकी दी है कि तुम्हारे परिवार को कौन बचाएगा , सलमान खान गुंडों और माफियाओं की तरह क्यों धमकी दे रहा है ..... । (अर्काइव ट्वीट)देखें ट्वीट:क्या है वायरल वीडियो का सच ?हमारी पड़ताल में सामने आया कि सलमान खान का यह वीडियो हाल का नहीं बल्कि 4 साल पुराना है। वीडियो कोरोना महामारी के समय का है, तब एक्टर ने लोगों से घर पर रहने और पूरी सावधानी बरतने की अपील की थी। पड़ताल के दौरान हमें यह वीडियो Sansad TV के आधिकारिक यूट्यूब चैनल पर मिला।देखें वीडियो:इस वीडियो को 16 अप्रैल 2020 को अपलोड किया गया था। कुल 2:16 मिनट के इस वीडियो में वायरल हिस्सा 1 मिनट 38 सेकेंड से सुना जा सकता है। स्पष्ट है कि सलमान खान को लेकर किया जा रहा दावा पूरी तरह से गलत और भ्रामक है।फेक न्यूज के खिलाफ हमारे साथ जुड़ें। कोई भी ऐसी सूचना जिस पर आपको संदेह हो तो हमें ईमेल करें @fakenewsexpose@dbcorp.in और वॉट्सऐप करें- 9201776050"
prediction = predict(article)
print(f"The article is classified as: {prediction}")

The article is classified as: fake


In [138]:
df["news"][9]

"क्या सलमान खान ने लॉरेंस को जवाब दिया है? दरअसल, पिछले दिनों हुए एनसीपी नेता बाबा सिद्दीकी के मर्डर की जिम्मेदारी लॉरेंस गैंग ने ली है। सलमान खान पहले से ही लॉरेंस गैंग के निशाने पर हैं।वेरिफाइड एक्स यूजर डॉ. शीतल यादव ने अपने ट्वीट में लिखा-सलमान खान ने साफ शब्दों में लॉरेंस को जवाब दिया है। 'भिड़ोगे तो मिट जाओगे'। (अर्काइव ट्वीट)देखें ट्वीट:खबर लिखे जाने तक डॉ. शीतल यादव के इस ट्वीट को 19 हजार लोग लाइक कर चुके थे वहीं, 3300 लोगों ने इसे रीपोस्ट किया था। एक्स पर डॉ. शीतल यादव को 24 हजार यूजर्स फॉलो करते हैं।एक्स हैंडल, बेवजह-के-ख़याल ने भी वायरल वीडियो को ट्वीट करते हुए लिखा- 'सलमान खान का पैगाम लॉरेंस बिश्नोई को'। (अर्काइव लिंक)देखें ट्वीट:वहीं, वेरिफाइड एक्स यूजर अभिषेक गुप्ता ने अपने ट्वीट में लिखा-सलमान खान ने अब लॉरेंस बिश्नोई को धमकी दी है कि तुम्हारे परिवार को कौन बचाएगा , सलमान खान गुंडों और माफियाओं की तरह क्यों धमकी दे रहा है ..... । (अर्काइव ट्वीट)देखें ट्वीट:क्या है वायरल वीडियो का सच ?हमारी पड़ताल में सामने आया कि सलमान खान का यह वीडियो हाल का नहीं बल्कि 4 साल पुराना है। व

In [140]:
# Save the model and tokenizer
model.save_pretrained("D:\Desktop\FakeNews\model")
tokenizer.save_pretrained("D:\Desktop\FakeNews\model")


('D:\\Desktop\\FakeNews\\model\\tokenizer_config.json',
 'D:\\Desktop\\FakeNews\\model\\special_tokens_map.json',
 'D:\\Desktop\\FakeNews\\model\\vocab.txt',
 'D:\\Desktop\\FakeNews\\model\\added_tokens.json')

In [ ]:
# TO Load it ,again .
model = BertForSequenceClassification.from_pretrained("D:\Desktop\FakeNews\model")
tokenizer = BertTokenizer.from_pretrained("D:\Desktop\FakeNews\model")